# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [5]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [7]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food']


In [8]:
# assign the uk_food database to a variable name
UK_Food = mongo['uk_food']

In [9]:
# review the collections in our new database
print(UK_Food.list_collection_names())

['establishments']


In [10]:
# assign the collection to a variable
establishments = UK_Food['establishments']

In [11]:
# review a document in the establishments collection
pprint(establishments.find_one())

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63d753563cf263aebad43877'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [53]:
# Create a dictionary for the new restaurant data
from bson import ObjectId

new_halal_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}


In [15]:
# Insert the new restaurant into the collection
UK_Food.establishments.insert_one(new_halal_restaurant)

In [60]:
# Check that the new restaurant was inserted
#query
new_halal_restaurant = {'BusinessName': 'Penang Flavours'}
#results
inserted_restaurant = establishments.find_one(new_halal_restaurant)
#pprint
pprint(inserted_restaurant)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63daa2dfcf97ab34571dc118'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [61]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
result = UK_Food.establishments.find_one({"BusinessType": "Restaurant/Cafe/Canteen"},{"BusinessTypeID": 1, "BusinessType": 1})
pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('63d753563cf263aebad43877')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [62]:
# Update the new restaurant with the correct BusinessTypeID
business_type_id = result['BusinessTypeID']
UK_Food.establishments.update_one({"BusinessName": "Penang Flavours"}, {"$set": {"BusinessTypeID": business_type_id}})

In [63]:
# Confirm that the new restaurant was updated
# by printing & visually checking the change in 'BusinessTypeID'
updated_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
updated_restaurant

#or using if/else condition, more convenient if you are checking multiple documents

# if updated_restaurant["BusinessTypeID"] == business_type_id:
#     print("BusinessTypeID is updated.")
# else:
#     print("No BusinessTypeID update was found.")

{'_id': ObjectId('63daa2dfcf97ab34571dc118'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [20]:
# Find how many documents have LocalAuthorityName as "Dover"
# dover_count = establishments.count_documents({"LocalAuthorityCode": "182"})
# or
dover_count = establishments.count_documents({'LocalAuthorityName': 'Dover'})
dover_count #994

994

In [21]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName': 'Dover'})

In [22]:
# Check if any remaining documents include Dover
dover_count = establishments.count_documents({'LocalAuthorityName': 'Dover'})
dover_count

0

In [13]:
# Check that other documents remain with 'find_one'
#39779 document(s) imported successfully plus 1 insertion, total is 39780

updated_count = establishments.count_documents({})
print(f'remaining documents: {updated_count}') #38786

#39780 (total docs) less 994 (dover_count) is 38786 (updated_count) 

# remaining_document = establishments.find_one({"LocalAuthorityName": {"$ne":"Dover"}})     or

remaining_document = establishments.find_one()
pprint(remaining_document)

remaining documents: 38786
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63d753563cf263aebad43877'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [49]:
# Change the data type from String to Decimal for longitude
decimal_longitude=establishments.update_many({},[{'$set':{'geocode.longitude':{'$toDecimal':'$geocode.longitude'}}}])
            
# Change the data type from String to Decimal for latitude 
decimal_latitude=establishments.update_many({},[{'$set':{'geocode.latitude':{'$toDecimal':'$geocode.latitude'}}}])                                              
  

In [50]:
# To convert the coordinates to float/numbers:

# Change the data type from Decimal to float for longitude
float_longitude = establishments.update_many({}, [{'$set': {'geocode.longitude': {'$toDouble': '$geocode.longitude'}}}])

# Change the data type from Decimal to float for latitude
float_latitude = establishments.update_many({}, [{'$set': {'geocode.latitude': {'$toDouble': '$geocode.latitude'}}}])

In [51]:
# Check that the coordinates are now numbers using for loop
for doc in establishments.find({}, {"geocode.latitude": 1, "geocode.longitude": 1}):
    print(f'latitude: {doc["geocode"]["latitude"]}, type: {type(doc["geocode"]["latitude"])}')
    print(f'longitude: {doc["geocode"]["longitude"]}, type: {type(doc["geocode"]["longitude"])}')

latitude: 51.083812, type: <class 'float'>
longitude: 1.195625, type: <class 'float'>
latitude: 51.085797, type: <class 'float'>
longitude: 1.194762, type: <class 'float'>
latitude: 51.086058, type: <class 'float'>
longitude: 1.196408, type: <class 'float'>
latitude: 51.08084, type: <class 'float'>
longitude: 1.188537, type: <class 'float'>
latitude: 51.08084, type: <class 'float'>
longitude: 1.188537, type: <class 'float'>
latitude: 51.08084, type: <class 'float'>
longitude: 1.188537, type: <class 'float'>
latitude: 51.08084, type: <class 'float'>
longitude: 1.188537, type: <class 'float'>
latitude: 51.0783519967076, type: <class 'float'>
longitude: 1.18590330311705, type: <class 'float'>
latitude: 51.0783519967076, type: <class 'float'>
longitude: 1.18590330311705, type: <class 'float'>
latitude: 51.0783519967076, type: <class 'float'>
longitude: 1.18590330311705, type: <class 'float'>
latitude: 51.0783519967076, type: <class 'float'>
longitude: 1.18590330311705, type: <class 'float'

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)




longitude: 0.0040261, type: <class 'float'>
latitude: 51.5388822, type: <class 'float'>
longitude: 0.0032957, type: <class 'float'>
latitude: 51.555776, type: <class 'float'>
longitude: 0.02041, type: <class 'float'>
latitude: 51.538856, type: <class 'float'>
longitude: 0.004389, type: <class 'float'>
latitude: 51.542115, type: <class 'float'>
longitude: 0.006349, type: <class 'float'>
latitude: 51.542115, type: <class 'float'>
longitude: 0.006349, type: <class 'float'>
latitude: 51.542115, type: <class 'float'>
longitude: 0.006349, type: <class 'float'>
latitude: 51.553105, type: <class 'float'>
longitude: 0.017262, type: <class 'float'>
latitude: 51.5419804, type: <class 'float'>
longitude: 0.0058643, type: <class 'float'>
latitude: 51.542454, type: <class 'float'>
longitude: 0.0065629, type: <class 'float'>
latitude: 51.6492118835449, type: <class 'float'>
longitude: 0.113893002271652, type: <class 'float'>
latitude: 51.5453605651855, type: <class 'float'>
longitude: 0.009088000282

In [52]:
#Visual check
#Check that the coordinates are now numbers
pprint(UK_Food.establishments.find_one())    #'geocode': {'latitude': Decimal128('51.083812'),
                                             # 'longitude': Decimal128('1.195625')},

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('63d753563cf263aebad43877'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': None